# "Image Matching Challenge 2023: The Unbearable Weight of the Bundle Adjustment"
> "3D reconstruction is harder than two view matching"
- toc: false
- image: images/pascal-cage.jpg
- branch: master
- badges: true
- comments: true
- hide: false
- search_exclude: false

## The Unbearable Weight of the Bundle Adjustment and $50K money prize

This year Image Matching Challenge introduced two big changes. First, we went from two-view matching to full Structure-from-Motion as a task. Actually, the multiview track was present in pre-Kaggle era of IMC -- in [2019](https://image-matching-workshop.github.io/leaderboard/), [2020](https://www.cs.ubc.ca/research/image-matching-challenge/2020/leaderboard/) and [2021](https://www.cs.ubc.ca/research/image-matching-challenge/2021/leaderboard/), so we kind of returned to the roots. 

Participants were given with the sets of images and output should be the cameras poses for all of them. 
Second, thanks to our sponsors -- Google, [Haiper](https://www.haiper.ai), and Kaggle itself, we were able to propose $50k prize fund. With the strict open license condition (MIT/Apache 2/etc) for the "in-money solution".

### 3D reconstruction is not cheap

While going from image pairs to image sets might seem a small change, it has a significant impact on cumpute requirements. For the two-view case, one can run almost everything on GPU, e.g. SuperPoint for feature detection, SuperGlue for image matching, or the LoFTR for detector-less image matching. The only CPU part is the RANSAC, which can take as little as [10ms per image pair for VSAC](https://arxiv.org/abs/2106.10240), so not a big deal.

For the multiview case, on the other hand, one needs to performs the bundle-adjustment, which is a CPU-heavy task. Kaggle virtual machines, in addition to that, offer only 2-core CPU, so the 3D reconstruction itself becomes the main computational bottleneck of the whole process. 

### Example solution

To provide the participants a headstart, we have worked with Kaggle engineers to include the [pycolmap](https://github.com/colmap/pycolmap) into the default Kaggle kernels. Based on it, we have provided an example submission, which uses local features included in [kornia](link) library: [LoFTR](https://zju3dv.github.io/loftr/), [DISK](https://github.com/cvlab-epfl/disk), and [KeyNet-AffNet-HardNet](https://kornia-tutorials.readthedocs.io/en/latest/_nbs/image_matching_adalam.html). 

All of them have Apache 2 license, and has shown a good performance in one of the previous IMCs: LoFTR was a part of [top solutions in 2022](https://www.kaggle.com/competitions/image-matching-challenge-2022/discussion/328854), [recap](https://ducha-aiki.github.io/wide-baseline-stereo-blog/2022/07/05/IMC2022-Recap.html), DISK - in [2020](https://www.cs.ubc.ca/research/image-matching-challenge/2020/leaderboard/), [recap](https://ducha-aiki.github.io/wide-baseline-stereo-blog/2021/05/14/IMC2020-competition-recap.html) and [2021](https://www.cs.ubc.ca/research/image-matching-challenge/2021/leaderboard/), and KeyNet-AffNet-HardNet - one of the leaders in the [original IJCV-2020 publication](https://arxiv.org/abs/2003.01587). 

The LoFTR example, however, was so heavy, that was causing time-out error without the modifications. 

## New datasets, new challenges: UAV-to-ground, day-night, repeated patterns, wiry objects, scale change

In 2023 we have prepared 3 new datasets - Heritage, Haiper and Urban.

Each dataset has been split into "training" -- public, and private part. The parts were geographically disjoint, but shared a similar nuisance factors. We haven't yet decided if we are going to release "hidden" part or not.


### Heritage 

This dataset features the high resolution photos of ancient buildings, taken with DLSR cameras from the ground, as well as UAV photos. Particular challenges are:

- large scale change (up to 20x), from that overview photo to the close-up of the small detail, together with in-plane rotation


![](2023-07-05-IMC2023-Recap_files/Dioscuri.png "Images of the Dioscuri temple from Heritage dataset")


- ["banana effect"](https://dev.epicgames.com/community/learning/tutorials/1xM4/capturing-reality-banana-effect-what-to-do-if-my-model-is-bent), when the flat surface is often reconstructed as curved due to low overlap between consecutive frames and slight misalignments, which accululate altogether.

![](2023-07-05-IMC2023-Recap_files/wall.png "3D model of the `wall` from Heritage dataset")

- high-resolution image processing. See above about the computational challenges of bundle adjustment and then multiply that by high resolution AND large image number. Also, the most of deep learning features (DISK, LoFTR) can easily give you CUDA OOM error when run on 20 Mp image pair. 


### Haiper

Haiper (training) dataset is similar to captures for NERFs - layered "dome" of cameras, going around some object. The object itself is often thin (bicycle) or textureless (statue). The test part of the dataset has very small number of images, making the viewpoint difference the biggest challenge. 

![](2023-07-05-IMC2023-Recap_files/att_00000.png "3D model 'bike' from the Haiper dataset")

If IMC-2023 have discovered and used "[Map-free Visual Relocalization](https://research.nianticlabs.com/mapfree-reloc-benchmark)" for training their pipelines, that would likely help them with Haiper dataset as well. However, everyone has missed this opportunity, including us - organizers. 


### Urban 

The Urban dataset is, probably, the most similar one to IMC2020 PhotoTourism dataset. It covers photos of buldings in city, similar to PhotoTourism. 

Here is the photos of the "Kyiv Puppet Theater" -- the easiest and public -- part of the dataset.

![image.png](2023-07-05-IMC2023-Recap_files/att_00001.png)


In addition to the day-night photos, the hidden part of the dataset features highly symmetrical objects, such as the photo I took yesterday in Český Krumlov. The camera poses are actually look into each other, so visual overlap is zero. The only chance of non-wrongly-matching them, is either considering all the photos altogether, or utilizing the background. None of the existing feature matching solutions does this, as far as I know.

![](2023-07-05-IMC2023-Recap_files/IMG_4345_IMG_4352_matches.png "SuperGlue matches for symmetrical structures, photo from Český Krumlov to illustrate the challenges of the Urban dataset")


## Findings from the competition


### There are no 3d reconstruction libraries besides colmap

At least, there are none, which you can easily compile on Kaggle kernel and then use from python notebook. I personally really hoped to see some global SfM solution like [Theia](http://theia-sfm.org) or maybe [OpenSfM](https://github.com/mapillary/OpenSfM) to appear among the top-solution,as it could provide a significant speed-up over the incremental SfM like Colmap. 
Python bindings and ease of compilation is crutial factor here. 


### No NERF-like or any other solution than classical SfM

Despite all the progress, it seems that if one doesn't have any additional information, such as RGBD, or initial ARKit cameras poses, the best thing one can do with a challenging image collection, is classical SfM. 
No [SparsePose](https://arxiv.org/abs/2211.16991), no [FORGE](https://arxiv.org/abs/2212.04492)


### Global descriptor-based co-visibility is hard

Given that exhaistive image matching grows quadratically with number of images, it is very tempting to filter out some of those image pairs, based on some kind of covisibility criterion. And the simplest/fastest to compute is global descriptor one - you get a single vector per image, calculate global image similarity and remove those image pairs, which similarity is below threshold. Or take the top-K most similar images. In fact, I have used such approach in our paper ["Efficient Initial Pose-graph Generation for Global SfM"](https://arxiv.org/abs/2011.11986). However, what worked on large (thousands) image collections with dense viewpoint coverage, as 1DSfM, does not work that well on sparse image collections with many close-ups, rotations and illumination changes. 

Some teams had moderate success with [NetVLAD](https://arxiv.org/abs/1511.07247) and [DINOv2](https://github.com/facebookresearch/dinov2) global descriptors, however, the best strategy, it seems, to use the two-view matching itself, but on a smaller resolution -- such as SuperGlue, or KeyNet-AffNet-HardNet-AdaLAM.


### PixelSfM is good idea, but needs improvements

Many participants has tried to improve the initial SfM camera poses by utilizing [PixelPerfectSfM](https://github.com/cvg/pixel-perfect-sfm) -- the feature-metric bundle adjustment. While it imrpoves results, it took a lot of time and especially memory, which rendered it unpractical for many teams. Another challenge is the package compilation itself, which is not easy either.

[1st-place team](https://www.kaggle.com/competitions/image-matching-challenge-2023/discussion/417407) proposed a novel version of the correspondence and poses refimenent instead - called [Detector-Free Structure from Motion](https://zju3dv.github.io/DetectorFreeSfM/).


![](2023-07-05-IMC2023-Recap_files/inbox-14597895-2b2b6c045d8a2dfa0a536090f025db02-main_fig.png "1st place solution, multiview refinement.")


### Rotation-invariance is important, but easy to achieve for SuperGlue and LoFTR

Just rotate one of the images 4 times and select the best matches. 
Here is the image of the [3rd place solution](https://www.kaggle.com/competitions/image-matching-challenge-2023/discussion/416918), explaining it all.

![](2023-07-05-IMC2023-Recap_files/inbox-9249230-1036693590b6655d8d48d588bfd69e9c-IMC-solution.png "3rd place solution, brute-force rotation estimation, high-res image tiling")


### Initial image pair setup in Colmap is suboptimal

Several teams reported that manually setting the image pair to start incremental reconstruction in Colmap improved results. One even can improve results by running the incremental reconstruction several times. 


### Many things do not work until they do

Many teams have reported that LoFTR or DKM doesn't work for them - but both LoFTR and DKM are part of top-5 solutions. Recent "SiLK" keypoints are reported as not working, but maybe nobody just found a proper way to use them.


### "Old" features are not done yet - KeyNetAffNet-HardNet solution.

5th place actually end up in money by using an modified [example submission](https://github.com/ducha-aiki/imc2023-kornia-starter-pack/blob/main/keynetaffnet-adalam-pycolmap-3dreconstruction.ipynb) with a classical pipeline using such ancient local features as DoG, Harris and GFTT, together with KeyNet (2019), coupled together with [local affine shape estimation](https://ducha-aiki.github.io/wide-baseline-stereo-blog/2020/07/17/affine-correspondences.html) by AffNet, patch descriptor HardNet and hadncrafted AdaLAM matcher. 

Basically, that is cleverly engineered submission of the off-the-shelf local features, available in [kornia](https://kornia.readthedocs.io/en/latest/feature.html).


![](2023-07-05-IMC2023-Recap_files/inbox-3964695-051e9413d0ce9cd8357723ba7efab1be-full_pipeline.png "5th place solution, classical affine features, off-the-shelf kornia. Candidate for 1st money prize")


![](2023-07-05-IMC2023-Recap_files/7rlp3j.jpg  "Clever engineering is all you need sometimes")


### Permissive license and faster SuperGlue is out - LightGlue


[LightGlue](https://arxiv.org/pdf/2306.13643.pdf) - the solution by 7th place team and likely 2nd money prize, presents a SuperGlue-like architecture with early-stopping for easy image pairs and bunch of training recipes.
It also uses somewhat unpopular, but well-performing [ALIKED local feature](https://github.com/Shiaoming/ALIKED)

![](2023-07-05-IMC2023-Recap_files/att_00003.png "The LightGlue architecture from the paper")


Here is the quite from the paper about tricks that matter:

>Since the depth maps of MegaDepth are often incomplete, we also label points with a large epipolar error as unmatch- able. Carefully tuning and annealing the learning rate boosts the accuracy. Training with more points also does: we use 2k per image instead of 1k. The batch size matters: we use gradient checkpointing [10] and mixed-precision to fit 32 image pairs on a single GPU with 24GB VRAM.


In addition to that, homography-pretraining is crucial for the Light/Super-Glue performance. 


### No lines, monodepth or semantic segmentation this year.

Actually I hoped to see [Limap](https://github.com/cvg/limap/tree/main) line SfM or some kind of monocular depth models used. Bad luck.



## Per-dataset results

### Heritage

The Heritage dataset results show the biggest variability among the methods/worst/best case performance.

![image.png](2023-07-05-IMC2023-Recap_files/att_00007.png)

### Urban

Because the main Urban dataset challenge -- repeated patterns and symmetrical structures -- cannot be even theoretically addressed by better two-view feature matching, it is the hardest dataset. No single team tried to work on this part, so surprise it is hard. 

![](2023-07-05-IMC2023-Recap_files/att_00006.png "Urban dataset is the hardest among 2023 ones")

### Haiper

The Haiper results are either hit or miss w/o much difference between methods.
![](2023-07-05-IMC2023-Recap_files/att_00005.png "Haiper scenes are like meeting the dinosaur: you either meet it, or not")

## Conclusion: SfM is far from solved

As in many other computer vision tasks, if something seems to be "solved", that is just because the datasets are old and obsolete. You still need a good dense capture to do the 3D reconstruction. Many images, many compute, or the additional data like inertial module/GPS, or skilled person to do the capture, is required. See you hopefully next year. 
